# Recommender System Projectdef recommendations(title):
    i_d=[]
    indices=pd.Series(df1.index,index=df1['title']).drop_duplicates()
    idx = indices[title]
    dis_scores = list(enumerate(model[idx]))
    dis_scores = sorted(dis_scores, key=lambda x: x[1], reverse=True)
    dis_scores = dis_scores[1:31]
    idn = [i[0] for i in dis_scores]
    final =df1.iloc[idn].reset_index()
    idn = [i for i in final['index']]
    for j in idn:
        if(j<15951):
            i_d.append(j)
    indices=pd.Series(df.index,index=df['title']).drop_duplicates()
    for i in range(1,8):
        if (idn):
            print(indices.iloc[i_d].index[i])
     


Let's load the libraries.This would be helpful to make our operations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV
import datetime
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)

Let's read our datasets, All the forms of dataset are distinct,we should need to group them.

In [ ]:
movies = pd.read_csv("../input/movielens-20m-dataset/movie.csv")
movies.head()

In [ ]:
tags = pd.read_csv("../input/movielens-20m-dataset/tag.csv")
tags.head()

In [ ]:
ratings = pd.read_csv("../input/movielens-20m-dataset/rating.csv")
ratings.head()

In [ ]:
links = pd.read_csv("../input/movielens-20m-dataset/link.csv")
links.head()

In [ ]:
genome_scores = pd.read_csv("../input/movielens-20m-dataset/genome_scores.csv")
genome_scores.head()

In [ ]:
genome_tags = pd.read_csv("../input/movielens-20m-dataset/genome_tags.csv")
genome_tags.head()

First we should look that shapes of the frames,this should be useful for us to concat the frames.

In [ ]:
print("Movies shape: ", movies.shape)
print("Tags shape: ", tags.shape)
print("Ratings shape: ", ratings.shape)
print("Links shape: ", links.shape)
print("Genome shape: ", genome_scores.shape)
print("Genome tags shape: ", genome_tags.shape)

Movies and links frames have same size observation, we should look their unique count,then we can concatenate them

In [ ]:
movies["movieId"].nunique()

In [ ]:
links["movieId"].nunique()

We now concatenate these movies and links dataframe as well.

In [ ]:
movies_df = pd.concat([movies, links], axis=1)
movies_df.head()

We combined the movies and links frames for now, later we can concateate the others, however we should start to observe the history and infrastucture of the dataframe.

# PART I - Review The Structure of Data

In [ ]:
movies_df.head()

Let's remove one of the duplicate columns.

In [ ]:
movies_df = movies_df.loc[:, ~movies_df.columns.duplicated()]

Let's look the observations and proeprties.

In [ ]:
movies_df.shape

Let's look the types and the information of the dataframe

In [ ]:
movies_df.info()

This dataset has integer, objects and float values relatively.Let's look the statistical description of the numeric values.

In [ ]:
movies_df.describe()

Control the nan values, when we concatenate we can faced with some of them.

In [ ]:
movies_df.isnull().sum()

We have approximately hundred **nan** values in the movies dataframe.Think about to fill or drop them by the approach of our real dataset.

In [ ]:
movies_df.corr()

Visualize the correlation between the properties of our dataframe.

In [ ]:
f, ax = plt.subplots(figsize=(15, 5))
sns.heatmap(movies_df.corr(), annot=True, ax=ax, cmap="Blues")

By looking the correlation graph, Imdb ID and tmbdID has a high correlation with 82%. Maybe we can fill the nan tmbsID by using ImdbID's. Let's change the column names,this makes more cleare visualization for making operations.

In [ ]:
columns = ["MovieID", "Title", "Genres", "IMDBID", "TMBDID"]
movies_df.columns = columns

# PART II - Review The Attributes of Dataframes

* Step I; We will look the movies_df.
* Step II; We will look the ratings.
* Step III; Merge ratings and movies_df for making one super dataframe.

## PART II. I - Movies_Df Reviews

In [ ]:
movies_df.head()

### MovieID

In [ ]:
movies_df["MovieID"].head()

In [ ]:
movies_df["MovieID"].value_counts()

In [ ]:
movies_df["MovieID"].nunique()

We can gentely say this column's all observations are unique and repeats only once.All the frequencies are 1.

### Title

In [ ]:
movies_df["Title"].head()

In [ ]:
movies_df["Title"].value_counts()

Some films are repeat more than once times.

In [ ]:
movies_df["Title"].nunique()

This means that could be one person can rates the different movies respectively.

In [ ]:
movies_df["Title"].describe().T

Let's look the top frequent values.**"Black Field (2009)"**

In [ ]:
movies_df[movies_df["Title"] == "Black Field (2009)"]

### Genres

In [ ]:
movies_df["Genres"].head()

In [ ]:
movies_df["Genres"].value_counts()

In [ ]:
movies_df[movies_df["Genres"] == "Drama"].head()

In [ ]:
movies_df["Genres"].nunique()

In [ ]:
movies_df["Genres"].unique()

In [ ]:
movies_df["Genres"].describe().T

### IMDBID

In [ ]:
movies_df["IMDBID"].head()

In [ ]:
movies_df["IMDBID"].value_counts()

In [ ]:
movies_df["IMDBID"].nunique()

In [ ]:
movies_df["IMDBID"].unique()

### TMBDID

In [ ]:
movies_df["TMBDID"].head()

In [ ]:
movies_df["TMBDID"].value_counts()

TMBDID could be samne if one film can released in different years even it is updated or not.This can effects the situation of ID.

In [ ]:
movies_df[movies_df["TMBDID"] == 141971.0]

In [ ]:
movies_df["TMBDID"].nunique()

We have some missing values in that dataframe, this column nan values should be checked.

In [ ]:
movies_df["TMBDID"].isnull().sum()

In [ ]:
movies_df[movies_df["TMBDID"].isna()]

Control the nan values distribution,maybe we can find some way to out.

In [ ]:
movies_df[movies_df["TMBDID"].isna()]["Genres"].value_counts()

In [ ]:
movies_df[movies_df["Genres"] == "Documentary"]

Found a way like the most frequenct nan values are gonna filled by the genres mean.By using interpolation technique, we can fill them respectively.

In [ ]:
movies_df.interpolate(method="linear", inplace=True)

Let's control the exitence of nan values.

In [ ]:
movies_df.isnull().sum()

## PART II. II - Ratings - Movie Df Merge Operation

We should merge movies_df and ratings, because they specifies the same meaning by ratings values.

In [ ]:
ratings.head()

Let's change the columns of ratings dataframe.

In [ ]:
ratings.columns = ["UserID", "MovieID", "Rating", "TimeStamp"]

In [ ]:
movie_df = movies_df.merge(ratings, on="MovieID")

In [ ]:
movie_df.head()

In [ ]:
tags.columns = ["UserID", "MovieID", "Tag", "TimeStamp"]

Movie dataframe and the tags dataframe could not merged cause of the size of the dataframe, soo we can really say this could not merged.

In [ ]:
movie_df.head()

## Rating

In [ ]:
movie_df["Rating"].head()

In [ ]:
movie_df["Rating"].value_counts()

Lots of users are voted as 4 stars for the films recommendation.Visualization of the stars frequency could be very helpful for us.

In [ ]:
plt.figure(figsize=(20, 5))
movie_df["Rating"].value_counts().plot(kind="bar", color="#f74d69")
plt.xlabel("Rating Keys", fontsize=15)
plt.ylabel("Rating Frequencies", fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Rating Keys and Values", fontsize=15)
plt.legend()
plt.show()

### TimeStamp

Timestamp values should be converted to datetime from integer based.Let's control the datatypes of the property.

In [ ]:
movie_df["TimeStamp"].head()

In [ ]:
movie_df["TimeStamp"].dtype

Finally, TimeStamp column has no meanings for us, we only want to review the rating not the timestamp,this would not give any tricks to traack the data.

In [ ]:
movie_df.drop(["TimeStamp"], axis=1, inplace=True)

#  PART III - Analysis

Let's make some questions and try to find the respectives by asked questions.
* Inspirations are described from the orginal site;
    * Which genres receive the highest ratings? How does this change over time?
    * Determine the temporal trends in the genres/tagging activity of the movies released?

In [ ]:
movie_df.head()

In [ ]:
movie_df["Genres"].unique()

In [ ]:
movie_df["Genres"].head(20)

In [ ]:
number_rating = movie_df.groupby("Title")["Rating"].count().reset_index()

In [ ]:
number_rating

In [ ]:
number_rating.rename(columns={"Title": "Title",
                              "Rating": "Number of Ratings"}, inplace=True)

In [ ]:
number_rating.head()

Let's merged the number rating and movie_df frames. Merging operation is made a super dataframe.

In [ ]:
df = movie_df.merge(number_rating, on="Title")
df.head()

We can limited the dataframe as they are greater than 1000 counts.The frmae should be simplified by decreases the observations amount by using these operations.

In [ ]:
df = df[df["Number of Ratings"] >= 1000]

In [ ]:
pivot_table = df.pivot_table(index="Title", columns="UserID", values="Rating")
pivot_table.head()

In [ ]:
pivot_table.fillna(0, inplace=True)

# PART IV - K Nearest Neighbors - Collaborative Filtering Analysis
<BR></BR>
this algorithms is a supervised machine learning algorithm.This is non parametric --> Not predicting anything any assumptions from the dataset, Lazy algorithm --> KNN does not have any training step.
This algorithm is used both classification and regression problems.
<BR></BR>

KNN works like the way of selecting a value for the real ideal value as K.Then we calculate the all distances between the K points and all observation points.After that, we can find the KNearestNeighbors to the new data points. For the classification problems, we count the number of data points in each category among the K values. For regression, value for the new datapoint will be the average of the K Neighbors.
<BR></BR>

Distance can be calculated using 
* "Euclidean distance", 
* "Manhattan distance", 
* "Hamming Distance", 
* "Minkowski Distance".
<BR></BR>

Euclidean distance is calculated by getting the square of differences of independent and dependent values and their predictions.
Manhattan distance is calculated by getting the absolute value of the differences between the actual and predicted values.
Hamming distance is calculated by grouping categoric values. If my catgeoric values are grouped as large and medium, it returns 1, otherwise large-large combination returns only 0.
Minkowski distance is calculated by grouping the absolute values of the differences between actual and predicted values.The total number of repeats time gives as a exponential.The results exponential is gonna 1/p too.
<BR></BR>
KNN provides pros and cons too.Let's dive into them. 
* PROS: 
    ** Simple and easy to implement.
    ** Non parametric - no assumptions about underlaying data.
    ** Both classification and regression.
    ** Training step is faster. 
* CONS:
    ** KNN is expensive for new point predictions.
    ** Sensitive to the outliers.
    ** Accuracy can be deviate from the target by the noise and irrelevant data.  
    

In [ ]:
movie_sparse = csr_matrix(pivot_table)

In [ ]:
model = NearestNeighbors(n_neighbors=5, algorithm="brute", metric="cosine")
model

In [ ]:
model.fit(movie_sparse)

In [ ]:
df.drop(columns=["Genres", "UserID", "Rating"],inplace=True)

In [ ]:
df1 = df.copy()
t_list = []
for i in df["Title"]:
    t_list.append(i.split(" (")[0])
df1["Title"] = t_list

In [ ]:
def get_recommendations(movie_name):
    distances, suggestions = model.kneighbors(
        pivot_table.loc[movie_name].values.reshape(1, -1), n_neighbors = 5)
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(pivot_table.index[i]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i,
                                                           pivot_table.index[suggestions.flatten()[i]],
                                                           distances.flatten()[i]))

In [ ]:
for i in range(0, len(pivot_table)):
    print("\n****************************\n")
    get_recommendations(pivot_table.index[i])
    print("\n****************************\n")

# PART V - Content-Based Algorithm

Let's remember our content-based algorithm. We will use TFID vectorizer for the content-based recommendation techniques.

In [ ]:
movie_df.head()

Let's use the command for the text formatter vectorizer in our kernel by using python command.

In [ ]:
pip install tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [ ]:
tfv = TfidfVectorizer(analyzer="word", ngram_range=(1, 1000), min_df=1)
x = tfv.fit_transform(movie_df["Genres"])

In [ ]:
model = sigmoid_kernel(x, x)

In [ ]:
df1=df.copy()
ti=[]
for i in df1["Title"]:
    ti.append(i.split(' (')[0])
df1["Title"]=ti

In [ ]:
def get_contentbasedrecommendations(title):
    i_d=[]
    indices=pd.Series(df1.index,index=df1["Title"]).drop_duplicates()
    idx = indices[title]
    dis_scores = list(enumerate(model[idx]))
    dis_scores = sorted(dis_scores, key=lambda x: x[1], reverse=True)
    dis_scores = dis_scores[1:31]
    idn = [i[0] for i in dis_scores]
    final =df1.iloc[idn].reset_index()
    idn = [i for i in final['index']]
    for j in idn:
        if(j<15951):
            i_d.append(j)
    indices=pd.Series(df.index,index=df["Title"]).drop_duplicates()
    for i in range(1,8):
        if (idn):
            print(indices.iloc[i_d].index[i])
     


In [ ]:
for i in range(0, len(pivot_table)):
    print("\n****************************\n")
    get_contentbasedrecommendations("Unforgettable (1996)")
    print("\n****************************\n")

# PART VI - Final Part
<br></br>
Finally, we wanted to built a recommendation system engine to predict the similar items of users prefers and we aimed with this collaborative item based filtering algorithm and also content-based filtering techniques.The all output gives us such a way to predict the future data by looking the previous data in movielens dataset.